In [ ]:
%matplotlib inline

# 可选内容：数据并行
**作者**: [Sung Kim](https://github.com/hunkim) 、 [Jenny Kang](https://github.com/jennykang)

在本教程中，我们将学会如何使用 `DataParallel` 操作多个 GPU 。

使用 PyTorch 操作 GPU 非常容易。你可以将模型放到 GPU 上：

```python
    device = torch.device("cuda:0")
    model.to(device)
```

然后，你可以将所有的 tensor 复制到 GPU 上：

```python
    mytensor = my_tensor.to(device)
```

请注意，仅仅调用 `my_tensor.to(device)` 会在 GPU 上返回一个新的 `my_tensor` 的副本，而不是重写 `my_tensor` 。
你需要将它分配给一个新的 tensor 并在 GPU 上使用这个新的 tensor。

在多个 GPU 上执行你的正向传播、反向传播过程是非常自然的。然而 PyTorch 默认只会使用一个 GPU 。
你可以通过使用 `DataParallel` 在多个 GPU 上并行运行你的模型，从而轻而易举地执行你的操作：

```python
    model = nn.DataParallel(model)
```

这是本教程的核心，我们接下来将讨论它的细节。


## 导入和参数
Imports and parameters

导入 PyTorch 模块，然后定义参数。


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# 参数和数据加载器
input_size = 5
output_size = 2

batch_size = 30
data_size = 100

设备


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## 虚拟数据集

构造一个虚拟（随机）数据集。你只需要实现 `__getitem__`


In [ ]:
class RandomDataset(Dataset):

    def __init__(self, size, length):
        self.len = length
        self.data = torch.randn(length, size)

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return self.len

rand_loader = DataLoader(dataset=RandomDataset(input_size, data_size),
                         batch_size=batch_size, shuffle=True)

## 简单模型

在这个 demo 中，我们的模型只有一个输入，执行线性操作，并给出输出。然而，你能在任何模型（ CNN 、 RNN 、 残差网络等等）上使用 `DataParallel` 。

我们已经在模型中放置了一个 print 语句来监视输入 tensor 和输出 tensor 的大小。
请注意在 batch rank 0 的时候打印了什么。


In [ ]:
class Model(nn.Module):
    # 我们的模型

    def __init__(self, input_size, output_size):
        super(Model, self).__init__()
        self.fc = nn.Linear(input_size, output_size)

    def forward(self, input):
        output = self.fc(input)
        print("\tIn Model: input size", input.size(),
              "output size", output.size())

        return output

## 创建模型并进行数据并行

这是本教程的核心部分。首先，我们需要创建一个模型实例，并检查我们是否有多个 GPU 。如果我们有多个 GPU ，我们就可以使用 `nn.DataParallel` 来包裹我们的模型。然后我们可以使用 `model.to(device)` 将我们的模型放到 GPU 上。


In [ ]:
model = Model(input_size, output_size)
if torch.cuda.device_count() > 1:
  print("Let's use", torch.cuda.device_count(), "GPUs!")
  # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
  model = nn.DataParallel(model)

model.to(device)

## 运行模型

现在我们可以看到输入 tensor 和输出 tensor 的 size 了。


In [ ]:
for data in rand_loader:
    input = data.to(device)
    output = model(input)
    print("Outside: input size", input.size(),
          "output_size", output.size())

## 结果

如果你没有 GPU 或者只有一个 GPU，当我们批量处理 30 个输入和 30 个输出时，模型将按照预期接收 30 个输入并输出 30 个输出 。但是如果你有多个 GPU 的话，你能看到如下的结果。

### 两个 GPU

如果你有两个 GPU ，你将看到：

```bash
    # on 2 GPUs
    Let us use 2 GPUs!
        In Model: input size torch.Size([15, 5]) output size torch.Size([15, 2])
        In Model: input size torch.Size([15, 5]) output size torch.Size([15, 2])
    Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
        In Model: input size torch.Size([15, 5]) output size torch.Size([15, 2])
        In Model: input size torch.Size([15, 5]) output size torch.Size([15, 2])
    Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
        In Model: input size torch.Size([15, 5]) output size torch.Size([15, 2])
        In Model: input size torch.Size([15, 5]) output size torch.Size([15, 2])
    Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
        In Model: input size torch.Size([5, 5]) output size torch.Size([5, 2])
        In Model: input size torch.Size([5, 5]) output size torch.Size([5, 2])
    Outside: input size torch.Size([10, 5]) output_size torch.Size([10, 2])
```

### 三个 GPU

如果你有三个 GPU ，你将看到：

```bash
    Let us use 3 GPUs!
        In Model: input size torch.Size([10, 5]) output size torch.Size([10, 2])
        In Model: input size torch.Size([10, 5]) output size torch.Size([10, 2])
        In Model: input size torch.Size([10, 5]) output size torch.Size([10, 2])
    Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
        In Model: input size torch.Size([10, 5]) output size torch.Size([10, 2])
        In Model: input size torch.Size([10, 5]) output size torch.Size([10, 2])
        In Model: input size torch.Size([10, 5]) output size torch.Size([10, 2])
    Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
        In Model: input size torch.Size([10, 5]) output size torch.Size([10, 2])
        In Model: input size torch.Size([10, 5]) output size torch.Size([10, 2])
        In Model: input size torch.Size([10, 5]) output size torch.Size([10, 2])
    Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
        In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
        In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
        In Model: input size torch.Size([2, 5]) output size torch.Size([2, 2])
    Outside: input size torch.Size([10, 5]) output_size torch.Size([10, 2])
```

### 八个 GPU

如果你有八个 GPU ，你将看到：

```bash
    Let us use 8 GPUs!
        In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
        In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
        In Model: input size torch.Size([2, 5]) output size torch.Size([2, 2])
        In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
        In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
        In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
        In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
        In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
    Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
        In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
        In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
        In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
        In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
        In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
        In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
        In Model: input size torch.Size([2, 5]) output size torch.Size([2, 2])
        In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
    Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
        In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
        In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
        In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
        In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
        In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
        In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
        In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
        In Model: input size torch.Size([2, 5]) output size torch.Size([2, 2])
    Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
        In Model: input size torch.Size([2, 5]) output size torch.Size([2, 2])
        In Model: input size torch.Size([2, 5]) output size torch.Size([2, 2])
        In Model: input size torch.Size([2, 5]) output size torch.Size([2, 2])
        In Model: input size torch.Size([2, 5]) output size torch.Size([2, 2])
        In Model: input size torch.Size([2, 5]) output size torch.Size([2, 2])
    Outside: input size torch.Size([10, 5]) output_size torch.Size([10, 2])
```


## 总结

DataParallel 自动将你的数据分割并发送作业订单给在多个 GPU 上的多个模型。
每一个模型做完它们的工作之后， DataParallel 收集并合并结果然后将结果返回给你。

了解更多信息，请点击[多 GPU 示例](https://pytorch.org/tutorials/beginner/former_torchies/parallelism_tutorial.html) 。
